<a href="https://colab.research.google.com/github/Inaracrewc/MovieLens_2M_1/blob/main/MovieLens_my.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [9]:
from fastbook import *
from fastai.collab import *
from fastai.tabular.all import *

In [10]:
import numpy as np
import pandas as pd
import os
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
path = '/content/gdrive/MyDrive/archive-4'

In [11]:
movies_df = pd.read_csv('/content/gdrive/MyDrive/archive-4/movie.csv')

In [12]:
print(movies_df.shape)
movies_df.head()

(27278, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
ratings_df = pd.read_csv("/content/gdrive/MyDrive/archive-4/rating.csv")
print(ratings_df.shape)
ratings_df.head()

(14585667, 4)


,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [14]:
ratings_df = ratings_df.drop('timestamp', 1)

In [ ]:
ratings_df.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [15]:
ratings = ratings_df.merge(movies_df)
print(ratings.shape)
ratings.head()

(14585667, 5)


,userId,movieId,rating,title,genres
0,1,2,3.5,Jumanji (1995),Adventure|Children|Fantasy
1,5,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
2,13,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
3,29,2,3.0,Jumanji (1995),Adventure|Children|Fantasy
4,34,2,3.0,Jumanji (1995),Adventure|Children|Fantasy


In [16]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,userId,title,rating
0,32341,Mr. Nobody (2009),4.0
1,91587,Annie Hall (1977),4.5
2,33323,"Hard Day's Night, A (1964)",3.0
3,10135,Dragonheart (1996),4.0
4,92194,"Return of the Pink Panther, The (1975)",4.0
5,90195,2001: A Space Odyssey (1968),4.5
6,55660,Pleasantville (1998),5.0
7,75054,"Man with Two Brains, The (1983)",5.0
8,57430,"Manchurian Candidate, The (2004)",4.5
9,99276,On the Waterfront (1954),3.0


In [17]:
dls.classes

{'title': ['#na#', '#chicagoGirl: The Social Network Takes on a Dictator (2013)', '$ (Dollars) (1971)', '$5 a Day (2008)', '$9.99 (2008)', '$ellebrity (Sellebrity) (2012)', "'71 (2014)", "'Hellboy': The Seeds of Creation (2004)", "'Human' Factor, The (Human Factor, The) (1975)", "'Neath the Arizona Skies (1934)", "'R Xmas (2001)", "'Round Midnight (1986)", "'Salem's Lot (2004)", "'Til There Was You (1997)", "'Twas the Night Before Christmas (1974)", "'burbs, The (1989)", "'night Mother (1986)", '(500) Days of Summer (2009)', '(A)sexual (2011)', '(Untitled) (2009)', '*batteries not included (1987)', '+1 (2013)', '...All the Marbles (California Dolls, The) (1981)', '...And God Spoke (1993)', '...And Justice for All (1979)', '...tick... tick... tick... (1970)', '.45 (2006)', '009 Re: Cyborg (2012)', '1 (2013)', '1, 2, 3, Sun (Un, deuz, trois, soleil) (1993)', '1-900 (06) (1994)', '10 (1979)', '10 000 timmar (2014)', '10 Items or Less (2006)', '10 Mountains 10 Years (2010)', '10 Questions 

In [18]:
embs = get_emb_sz(dls) #recommended sizes for embedding matrices for your data
embs

[(100741, 600), (24851, 463)]

In [19]:
class CollabNN(Module):# deep learning collaborative filtering model. the two embedding matrices can have different sizes (i.e., different numbers of latent factors)
    def __init__(self, user_sz, item_sz, y_range=(0,5.5), n_act=100):
        self.user_factors = Embedding(*user_sz)
        self.item_factors = Embedding(*item_sz)
        self.layers = nn.Sequential(
            nn.Linear(user_sz[1]+item_sz[1], n_act),
            nn.ReLU(),
            nn.Linear(n_act, 1))
        self.y_range = y_range
        
    def forward(self, x):
        embs = self.user_factors(x[:,0]),self.item_factors(x[:,1])
        x = self.layers(torch.cat(embs, dim=1))
        return sigmoid_range(x, *self.y_range)

In [20]:
model = CollabNN(*embs)

In [ ]:
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(3, 1e-3, wd=0.01)

epoch,train_loss,valid_loss,time


In [ ]:
# failed to train with DotProductBias

In [ ]:
x,y = dls.one_batch()
x.shape

torch.Size([64, 2])

In [30]:
n_users  = len(dls.classes['userId'])
n_movies = len(dls.classes['title'])
n_factors = 5   # attribute to each of our users and each of our movies a random vector of a certain length (here, n_factors=5)

user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)

In [ ]:
class T(Module):
    def __init__(self): self.a = nn.Parameter(torch.ones(3))

L(T().parameters())

(#1) [Parameter containing:
tensor([1., 1., 1.], requires_grad=True)]

In [ ]:
def create_params(size):    
  return nn.Parameter(torch.zeros(*size).normal_(0, 0.01))

In [ ]:
class DotProductBias(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
        self.user_factors = create_params([n_users, n_factors])
        self.user_bias = create_params([n_users])
        self.movie_factors = create_params([n_movies, n_factors])
        self.movie_bias = create_params([n_movies])
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors[x[:,0]]
        movies = self.movie_factors[x[:,1]]
        res = (users*movies).sum(dim=1)
        res += self.user_bias[x[:,0]] + self.movie_bias[x[:,1]]
        return sigmoid_range(res, *self.y_range)

In [ ]:
learn = collab_learner(dls, n_factors=50, y_range=(0, 5.5))

In [ ]:
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.976915,0.955184,25:13
1,0.925292,0.953772,25:24
2,0.929148,0.926357,26:41


KeyboardInterrupt: ignored

In [ ]:
SaveModelCallback()

SaveModelCallback

In [ ]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(100741, 50)
  (i_weight): Embedding(24851, 50)
  (u_bias): Embedding(100741, 1)
  (i_bias): Embedding(24851, 1)
)

In [ ]:
movie_bias = learn.model.i_bias.weight.squeeze()
idxs = movie_bias.argsort(descending=True)[:5]
[dls.classes['title'][i] for i in idxs]

['Shawshank Redemption, The (1994)',
 "Schindler's List (1993)",
 'Godfather, The (1972)',
 'Usual Suspects, The (1995)',
 'Matrix, The (1999)']

In [ ]:
movie_factors = learn.model.i_weight.weight
idx = dls.classes['title'].o2i['Return of the Pink Panther, The (1975)']
distances = nn.CosineSimilarity(dim=1)(movie_factors, movie_factors[idx][None])
idx = distances.argsort(descending=True)[1]
dls.classes['title'][idx]

'Irma la Douce (1963)'

In [ ]:
learn.save('movie-reviews')

Path('models/movie-reviews.pth')

In [ ]:
item_name = title

NameError: ignored

In [ ]:
g = ratings.groupby(title)['rating'].count()
top_movies = g.sort_values(ascending=False).index.values[:1000]
top_movies[:10]

movie_bias = learn.bias(top_movies,is_item=True)
ratings = [(b,i)for i,b in zip(top_movies,movie_bias)]

sorted(ratings) [:15]

In [ ]:
model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat(), metrics=accuracy)
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time


AssertionError: ignored

In [ ]:
inputs = torch.rand(size=(4, 4), dtype=torch.float32)